# Feature creation
Here I will create features such as signals (buy/sell) and market condition (bearish, bullish, sideways)

### Signals
**Buy:** If the price increases by 2% within 6 candlesticks (24h on a 4h timeframe)
**Sell:** Stop-loss (decreases by) 2% and take-profit (increases by) 4%

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("../Data/ADA/ADA_USDT_4h.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df.set_index("datetime", inplace=True)

df.head(5)

,open,high,low,close,volume
datetime,,,,,
2021-10-22 00:00:00,2.141,2.194,2.133,2.185,10918.3
2021-10-22 04:00:00,2.185,2.202,2.185,2.198,26091.8
2021-10-22 08:00:00,2.198,2.208,2.182,2.192,24092.3
2021-10-22 12:00:00,2.192,2.201,2.147,2.151,30423.4
2021-10-22 16:00:00,2.151,2.183,2.142,2.165,17709.4


In [5]:
look_ahead_candles = 6
percentage_threshold = 2
take_profit_threshold = 4 
stop_loss_threshold = -2

df['buy_signal'] = 0
df['sell_signal'] = 0

in_position = False
buy_price = None 

for index in range(len(df) - look_ahead_candles):
    current_close = df.at[df.index[index], 'close']
    
    if in_position:
        # Look for a sell only if we're in a position
        pct_change_since_buy = ((current_close - buy_price) / buy_price) * 100
        
        # take-profit or stop-loss
        if pct_change_since_buy >= take_profit_threshold or pct_change_since_buy <= stop_loss_threshold:
            df.at[df.index[index], 'sell_signal'] = 1
            in_position = False 
    else:
        # Look for a buy only if we're not in a position
        max_future_close = df['close'][index + 1:index + 1 + look_ahead_candles].max()
        max_pct_change = ((max_future_close - current_close) / current_close) * 100

        if max_pct_change >= percentage_threshold:
            df.at[df.index[index], 'buy_signal'] = 1
            buy_price = current_close
            in_position = True

print(df["buy_signal"].value_counts())
print(df["sell_signal"].value_counts())

buy_signal
0    4357
1     366
Name: count, dtype: int64
sell_signal
0    4358
1     365
Name: count, dtype: int64


In [6]:
df.to_csv("ADA_USDT_Signals.csv")

Because of the poor result in the file feature_selection_signals, I will try a different way of generating signals, without stop/loss and take/profit, and without restrictions of having buy signals be skipped if it's already in position. This means the trading strategy will have to be employed later in the process, but may improve the results for the model.

In [7]:
df = pd.read_csv("../Data/ADA/ADA_USDT_4h.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df.set_index("datetime", inplace=True)

df.head(5)

,open,high,low,close,volume
datetime,,,,,
2021-10-22 00:00:00,2.141,2.194,2.133,2.185,10918.3
2021-10-22 04:00:00,2.185,2.202,2.185,2.198,26091.8
2021-10-22 08:00:00,2.198,2.208,2.182,2.192,24092.3
2021-10-22 12:00:00,2.192,2.201,2.147,2.151,30423.4
2021-10-22 16:00:00,2.151,2.183,2.142,2.165,17709.4


In [9]:
look_ahead_candles = 6
percentage_threshold = 2

df['buy_signal'] = 0

for index in range(len(df) - look_ahead_candles):
    current_close = df.at[df.index[index], 'close']

    max_future_close = df['close'][index + 1:index + 1 + look_ahead_candles].max()
    
    max_pct_change = ((max_future_close - current_close) / current_close) * 100
    
    if max_pct_change >= percentage_threshold:
        df.at[df.index[index], 'buy_signal'] = 1

print(df["buy_signal"].value_counts())

df.to_csv("ADA_USDT_Signals2.csv")

buy_signal
0    3009
1    1714
Name: count, dtype: int64


In [3]:
# Recent data
df = pd.read_csv("../Data/ADA/ADA_USDT_recent.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df.set_index("datetime", inplace=True)

look_ahead_candles = 6
percentage_threshold = 2

df['buy_signal'] = 0

for index in range(len(df) - look_ahead_candles):
    current_close = df.at[df.index[index], 'close']

    max_future_close = df['close'][index + 1:index + 1 + look_ahead_candles].max()
    
    max_pct_change = ((max_future_close - current_close) / current_close) * 100
    
    if max_pct_change >= percentage_threshold:
        df.at[df.index[index], 'buy_signal'] = 1

print(df["buy_signal"].value_counts())

df.to_csv("ADA_USDT_Recent_Signals2.csv")

buy_signal
0    385
1    244
Name: count, dtype: int64
